In [ ]:
# dependencies

import gzip
from pickle import dump, load

import numpy as np
import pandas as pd

from switch import mcem
from switch.models import tanh

In [ ]:
# configure algorithm

n_iter = 100
n_init = 100
n_thin_mcmc = 10
init_scale_h = 2
learning_rate= 1.25
pr_portkey_thi = 1e-3
seed = 0

# configure prior

prior_shape_jumps = 1
prior_rate_jumps = 48

In [ ]:
# import data

f109 = pd.read_csv('paper/f109.csv', sep=',', parse_dates=['date']).iloc[:, 1:]
f109

In [ ]:
# select 2009 data

t = f109.cumTime.values[f109.date.dt.year == 2009]
vt = f109.centerE.values[f109.date.dt.year == 2009]

In [ ]:
# set up estimator

estimates = []
estimator = mcem.maximize_posterior(
    t, 
    vt, 
    2, 
    prior_shape_jumps, 
    prior_rate_jumps, 
    tanh.mod, 
    np.random.default_rng(seed), 
    init_scale_h=init_scale_h,
    learning_rate=1.25,
    n_init=n_init, 
    n_thin_mcmc=n_thin_mcmc)

In [ ]:
# run maximizer

for _ in range(n_iter):
    estimate = next(estimator)
    estimates.append(estimate[:3])

In [ ]:
# dump to disc

state = {k: v for k, v in estimator.gi_frame.f_locals.items() if k in
         ('thi', 'lam', 'hh', 'zz', 't', 'vt', 'hyper_lam', 'node_sampler', 'ctrl', 'ome')}
with gzip.open('paper/f109_mcem_state.pkl', 'wb') as f:
    dump(state, f)
with gzip.open('paper/f109_mcem_samples.pkl', 'wb') as f:
    dump(estimates, f)

In [ ]:
# to restart from saved state

# with gzip.open('paper/f109_mcem_state.pkl', 'rb') as f:
#     state = load(f)
# with gzip.open('paper/f109_mcem_samples.pkl', 'rb') as f:
#     estimates = load(f)

# estimator = mcem.resume_maxim(**state, mod=tanh.mod)